In [1]:
# Импорт цен акций за указанный период
import urllib.request
import datetime
import time
import math
import random
import nltk
import os
import string
import pandas as pd
import numpy as np
import scipy.stats as sps
import warnings
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings('ignore')
nltk.download('punkt')
# Тикеры акций и их id в системе Finam
tickers={'DJ.W1IDU':20447,'DJ.W1FIN':20459,'NASDAQCOMP':82075,'EURUSD':83,'USDCNY':83225,'INDUSDX':438796,'NYMEX.NG':18949,'ICE.BRN':19473,'GDAX.BCH-USD':491809,'comex.SI':18952, 'comex.GC':18953, 'USDCB':82485, 'D&J-IND':91, 'LME.Nickel':18932,'LME.Alum':18930, 'RTSI':95,'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001,'FIVE':7,'DSKY':8,'SFIN':9,'RNFT':9}
# Доп.признаки
extra=['DJ.W1FIN','DJ.W1IDU','NASDAQCOMP','EURUSD','USDCNY','NYMEX.NG','INDUSDX','ICE.BRN', 'LME.Alum','LME.Nickel','comex.GC','comex.SI','D&J-IND']
# Акции, входящие в индекс RTSI
RTSI=['GAZP','SBER','SBERP','LKOH','GMKN','YNDX','NVTK','TATN','TATNP','ROSN','SNGS','SNGSP','MGNT','FIVE','MTSS','POLY','ALRS','CHMF','PLZL','IRAO','NLMK','VTBR','MOEX','PHOR','TRNFP','MAGN','RTKM','RUAL','AFLT','PIKK','HYDR','FEES','AFKS','LSRG','CBOM','UPRO','DSKY','LNTA','SFIN','RNFT','MVID']
#Перевод массива строк в дату
def array2date(arr):
    new_arr=list()
    for item in arr:
        if len(item)==6:
            a=int(item[0:2])
            b=int(item[2:4])
            c=2000+int(item[4:])
            new_arr.append(datetime.date(c,b,a))
        elif len(item)==8:
            a=int(item[6:])
            b=int(item[4:6])
            c=2000+int(item[2:4])
            new_arr.append(datetime.date(c,b,a))
    return new_arr
# Формируем дату и время
def dtformed(d,mn,y,h=0,m=0,s=0):
    return datetime.datetime(y,mn,d,h,m,s)


# Возвращаем ID инструмента по его имени 
def getID(name):
    return tickers[name]
def load(name, dtstart=datetime.datetime.today(), 
         dtfin=datetime.datetime.today(),step=8, 
         market=1, MSOR=1, mstimever=0, sourse='finam', moment='close'):
    # market - инструмент
    # 0 — начала свечи, 1 — окончания свечи
    # mstimever - не московское — mstimever=0; московское — mstime='on', mstimever='1'
    #  период котировок (тики, 1 мин., 5 мин., 10 мин., 15 мин., 30 мин., 1 час, 1 день, 1 неделя, 1 месяц)
    id =getID(name) # id инстурмента
    e = '.txt' # расширение получаемого файла: .txt либо .csv
    yf = str(dtstart.year)
    yt = str(dtfin.year)
    month_start = str(dtstart.month)
    day_start = str(dtstart.day)
    month_end = str(dtfin.month)
    day_end = str(dtfin.day)
    dtf = '3' # формат даты (1 — ггггммдд, 2 — ггммдд, 3 — ддммгг, 4 — дд/мм/гг, 5 — мм/дд/гг)
    tmf = '1' # формат времени (1 — ччммсс, 2 — ччмм, 3 — чч: мм: сс, 4 — чч: мм)
    sep = '5' # параметр разделитель полей (1 — запятая (,), 2 — точка (.), 3 — точка с запятой (;), 4 — табуляция (»), 5 — пробел ( ))
    sep2 = '1' # параметр разделитель разрядов (1 — нет, 2 — точка (.), 3 — запятая (,), 4 — пробел ( ), 5 — кавычка ('))
    datf = '1' # Перечень получаемых данных (1-6)
    at = '0' # добавлять заголовок в файл (0 — нет, 1 — да)

    year_start = yf[2:]
    year_end = yt[2:]
    mf = (int(month_start.replace('0', ''))) - 1
    mt = (int(month_end.replace('0', ''))) - 1
    df = (int(day_start.replace('0', ''))) - 1
    dt = (int(day_end.replace('0', ''))) - 1
    
    user_agent = 'Chrome/98.0.4758.102 Safari/537.36'
    # Запрос
    headers={'User-Agent':user_agent,} 
    
    site='http://export.finam.ru/' + str(name) + '_' + str(year_start) + str(month_start) + str(
            day_start) + '_' + str(year_end) + str(month_end) + str(day_end) + str(e) + '?market=' + str(
            market) + '&em=' + str(id) + '&code=' + str(name) + '&apply=0&df=' + str(df) + '&mf=' + str(
            mf) + '&yf=' + str(yf) + '&from=' + str(day_start) + '.' + str(month_start) + '.' + str(yf) + '&dt=' + str(
            dt) + '&mt=' + str(mt) + '&yt=' + str(yt) + '&to=' + str(day_end) + '.' + str(month_end) + '.' + str(
            yt) + '&p=' + str(step) + '&f=' + str(name) + '_' + str(year_start) + str(month_start) + str(
            day_start) + '_' + str(year_end) + str(month_end) + str(day_end) + '&e=' + str(e) + '&cn=' + str(
            name) + '&dtf=' + str(dtf) + '&tmf=' + str(tmf) + '&MSOR=' + str(MSOR) + '&mstimever=' + str(
            mstimever) + '&sep=' + str(sep) + '&sep2=' + str(sep2) + '&datf=' + str(datf) + '&at=' + str(at)
    #Пока не удалось получить данные, пробуем это сделать
    errors=0
    while ('data' not in locals()):
        try:
            request=urllib.request.Request(site, None, headers) #The assembled request
            response = urllib.request.urlopen(request) 
            data = response.read() # The data u need
            #print(data)
        except urllib.error.HTTPError as e:
            errors+=1

    # Выделение нужных данных
    
    content = str(data)
    #print(content)
    stokelist=content.split(r'\r\n')
    n=len(stokelist)
    price_str=list()
    dates_str=list()
    for k in range(n-1):
        stokelist1=stokelist[k].split(' ')
        price_str.append(stokelist1[7])
        dates_str.append(stokelist1[2])
    dates=array2date(dates_str)
    prices=list()
    for item in price_str:
        prices.append(float(item))      
    return prices,dates

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Сбор данных по месяцам

In [2]:
feature_names=RTSI+extra+['RTSI']
n_start=dtformed(3,1,2016)
n_finish=dtformed(31,12,2019)
p_start=dtformed(1, 12, 2015)
p_finish=dtformed(31, 12, 2015)

In [3]:
def month_transform(arr):
    year1=arr[1][0].year
    year2=arr[1][-1].year
    num_months=(year2-year1+1)*12
    months=[[] for i in range(num_months)]
    for i in range(len(arr[0])):   
        k=(arr[1][i].year-year1)*12+arr[1][i].month-1
        months[k].append(arr[0][i])
    month_data=[0 for i in range(len(months))]
    for i in range(len(month_data)):
        month_data[i]=round(np.mean(months[i]),3)
    return np.reshape(np.array(month_data),(-1,1))    

In [4]:
targets1=month_transform(load("RTSI", n_start, n_finish))

In [5]:
def arr_to_mtrx(arr):
    v=arr[0]
    for i in range(1,len(arr)):
        v=np.hstack((v,arr[i]))
    return v
#Данные по акицям, дополнительным признакам, RTSI
stocks_arr1=list()
for item in RTSI:
    stocks_arr1.append(month_transform(load(item, n_start, n_finish)))
addstocks_arr1=list()
for item in extra:
    addstocks_arr1.append(month_transform(load(item, n_start, n_finish)))
addstocks1=arr_to_mtrx(addstocks_arr1)
stocks1=arr_to_mtrx(stocks_arr1)
#Объединяем массивы с данными по акциям RTSI и массивы с данными доп.признаков
df1=pd.DataFrame(np.hstack((stocks1,addstocks1,targets1)), columns=RTSI+extra+['RTSI'])

# Сохраняем данные

In [6]:
df1.to_csv("Month_Data.csv",index=False)

# Объединяем данные по неделям

In [7]:
def num_zeros(arr):
    zeros=0
    for i in range(len(arr)):
        if arr[i]==0:
            zeros+=1
    return zeros
def the_mean(arr):
    if len(arr)-num_zeros(arr)==0:
        return 0
    else:
        return round(sum(arr)/(len(arr)-num_zeros(arr)),3)
def week_lists(arr,date1,date2):
    inf=arr[0]
    date=arr[1]
    date1=datetime.date(date1.year,date1.month,date1.day)
    date2=datetime.date(date2.year,date2.month,date2.day)
    delta=(date2-date1).days+1
    new_arr=[0 for i in range(delta)]
    for i in range(len(inf)):
        if  (date[i]-date1).days<len(new_arr):
            new_arr[(date[i]-date1).days]=inf[i]
    week_arr=[]
    week_arr.append(new_arr[0:7+(delta%7)])
    for i in range((delta//7)-1):
        week_arr.append(new_arr[7+(delta%7)+i*7:7+(delta%7)+i*7+7])
    week_data=np.zeros((len(week_arr),1),dtype=np.float64)
    for i in range(len(week_arr)):
        week_data[i,0]=the_mean(week_arr[i])
    return week_data
def filler(arr):
    for i in range(0, arr.shape[0]-1):
        if arr[i,0]!=0 and arr[i+1,0]==0:
            arr[i+1,0]=arr[i,0]
    return arr   

In [8]:
a=datetime.date(2016,1,1)
b=datetime.date(2019,11,30)
targets2=filler(week_lists(load('RTSI',n_start,n_finish),a,b))

In [9]:
#Данные по акицям, дополнительным признакам, RTSI
stocks_arr2=list()
for item in RTSI:
    stocks_arr2.append(filler(week_lists(load(item,n_start,n_finish),a,b)))
addstocks_arr2=list()
for item in extra:
    addstocks_arr2.append(filler(week_lists(load(item,n_start,n_finish),a,b)))
#Объединяем массивы с данными по акциям
def arr_to_mtrx(arr):
    v=arr[0]
    for i in range(1,len(arr)):
        v=np.hstack((v,arr[i]))
    return v
addstocks2=arr_to_mtrx(addstocks_arr2)
stocks2=arr_to_mtrx(stocks_arr2)
df2=pd.DataFrame(np.hstack((stocks2,addstocks2,targets2)),columns=RTSI+extra+['RTSI'])

In [10]:
df2.to_csv("Week_Data.csv",index=False)

# Загружаем датасет с текстами

In [11]:
lenta=pd.read_csv('lenta-ru-news.csv')

# Считаем число пропусков

In [12]:
lenta.isna().sum()

url          0
title        0
text         5
topic    62002
tags     27219
date         0
dtype: int64

# Фильтруем датафрейм

In [13]:
#Отбрасываем столбцы, не представляющие интерес, удаляем данные с пропусками, берем период с 2016 по 2019 год
lenta.drop(columns = ['url', 'topic'],axis = 1, inplace=True)
lenta["date"] = pd.to_datetime(lenta["date"],format="%Y/%m/%d")
lenta=lenta[lenta['date']>pd.Timestamp('2015-12-31 00:00:00')]
lenta=lenta[lenta['date']<pd.Timestamp('2019-12-01 00:00:00')]
lenta=lenta.dropna()
lenta.reset_index(drop=True, inplace=True)

In [14]:
lenta.head(10)

,title,text,tags,date
0,В Техасе разрешили открыто носить оружие,В американском штате Техас владельцам огнестре...,Политика,2016-01-01
1,Запрет туркам вести строительство в России вст...,"Постановление правительства, запрещающее турец...",Госэкономика,2016-01-01
2,Вертолет Bell совершил жесткую посадку в Росто...,В Аксайком районе Ростовской области совершил ...,Происшествия,2016-01-01
3,В Новом Орлеане неизвестный расстрелял посетит...,В Новом Орлеане неизвестный в ночь на 1 января...,Происшествия,2016-01-01
4,Путин подписал указ о транзите грузов из Украи...,Президент Владимир Путин подписал указ о мерах...,Политика,2016-01-01
5,Сенатор сообщил об угрозе терактов из-за отмен...,Член комитета Совфеда по международным делам И...,Политика,2016-01-01
6,Косачев назвал сближение Европы и России тренд...,Глава комитета Совета Федерации по международн...,Политика,2016-01-01
7,Подросток погиб на пожаре после запуска фейерв...,"Подросток стал жертвой пожара, произошедшего в...",Общество,2016-01-01
8,В МВД заявили об отсутствии серьезных нарушени...,Новогодняя ночь в России прошла спокойно. Така...,Общество,2016-01-01
9,В новогоднюю ночь во Франции сожгли более 800 ...,Во Франции в новогоднюю ночь в общей сложности...,Происшествия,2016-01-01


# Определяем темы новостей, выбираем интересующие

In [15]:
lenta.tags.unique()

array(['Политика', 'Госэкономика', 'Происшествия', 'Общество', 'Звери',
       'Украина', 'Кино', 'Прибалтика', 'Музыка', 'Рынки', 'Конфликты',
       'Наука', 'Деньги', 'Футбол', 'Люди', 'Преступность', 'Интернет',
       'Coцсети', 'Летние виды', 'Криминал', 'Следствие и суд',
       'Средняя Азия', 'Бизнес', 'Техника', 'Еда', 'Все', 'Мир',
       'Бокс и ММА', 'Вещи', 'Россия', 'Явления', 'Гаджеты', 'Пресса',
       'Полиция и спецслужбы', 'Мировой бизнес', 'Достижения', 'Книги',
       'Стиль', 'Деловой климат', 'ТВ и радио', 'Космос', 'Инструменты',
       'Молдавия', 'Зимние виды', 'Закавказье', 'Регионы', 'Квартира',
       'События', 'Движение', 'Офис', 'Часы', 'Город', 'Игры', 'Москва',
       'Искусство', 'Мнения', 'Финансы компаний', 'Дача', 'Белоруссия',
       'Софт', 'Театр', 'История', 'Реклама', 'Вкусы', 'Внешний вид',
       'Страноведение', 'Автобизнес', 'Аналитика рынка', 'Производители',
       'Туризм', 'Госрегулирование', 'Выборы', 'Экология', 'Оружие',
       'Ме

In [16]:
useful_tags=['Госэкономика','Рынки','Бизнес','Экономика','Финансы компаний','Деловой климат','Мировой бизнес']
#useful_tags1=['Госэкономика','Рынки','Бизнес','Россия','Экономика','Финансы компаний','Деловой климат','Мировой бизнес','Госрегулирование']

In [17]:
f_lenta=lenta[lenta['tags'].isin(useful_tags)]
f_lenta.reset_index(drop=True,inplace=True)

# Разделяем датасет на недели

In [18]:
date_len=(datetime.datetime(2019,11,30)-datetime.datetime(2016,1,1)).days+1
weeks=list()
weeks.append(lenta.date.unique()[0:7+date_len%7])
for i in range((date_len//7)-1):
    weeks.append(lenta.date.unique()[7+date_len%7+i*7:7+date_len%7+i*7+7])
week_sets=list()
for i in range(len(weeks)):
    week_sets.append(f_lenta[f_lenta['date'].isin(weeks[i])])
for i in range(len(week_sets)):
    week_sets[i].reset_index(drop=True,inplace=True)

# Разделяем датасет на месяцы

In [19]:
months=list()
long_year=[31,29,31,30,31,30,31,31,30,31,30,31]
short_year=[31,28,31,30,31,30,31,31,30,31,30,31]
month_len=(long_year+3*short_year)[:-1]
the_date=0
for i in range(len(month_len)):
    months.append(lenta.date.unique()[the_date:the_date+month_len[i]])
    the_date=the_date+month_len[i]    
month_sets=list()
for i in range(len(months)):
    month_sets.append(f_lenta[f_lenta['date'].isin(months[i])])
for i in range(len(month_sets)):
    month_sets[i].reset_index(drop=True,inplace=True)

Сокращенный список датасетов, за каждую неделю берем первые 10 новостей, если их больше 10


In [20]:
shortweek_sets=week_sets.copy()
for i in range(len(week_sets)):
    if len(shortweek_sets[i])>10:
        shortweek_sets[i]=shortweek_sets[i][:11]

Сокращенный список датасетов, за каждый месяц берем первые 40 новостей, если их больше 40


In [21]:
shortmonth_sets=week_sets.copy()
for i in range(len(week_sets)):
    if len(shortmonth_sets[i])>10:
        shortmonth_sets[i]=shortmonth_sets[i][:11]

Объединяем тексты внутри минимальных временных отрезков(недель, месяцев)

In [22]:
def text_union(arr):
    a=str()
    for i in range(len(arr['text'])):
        a+=arr['text'][i]
    return a

In [23]:
month_union_texts=pd.DataFrame(columns=['date','text'])
for i in range(len(shortmonth_sets)):
    shortmonth_sets[i].reset_index(drop=True,inplace=True)
    new_row={'date':shortmonth_sets[i]['date'][0],'text':text_union(shortmonth_sets[i])}
    month_union_texts=month_union_texts.append(new_row,ignore_index=True)

In [24]:
week_union_texts=pd.DataFrame(columns=['date','text'])
for i in range(len(shortweek_sets)):
    shortweek_sets[i].reset_index(drop=True,inplace=True)
    new_row={'date':shortweek_sets[i]['date'][0],'text':text_union(shortweek_sets[i])}
    week_union_texts=week_union_texts.append(new_row,ignore_index=True)

# Сохраняем тексты

In [25]:
month_union_texts.to_csv('month_texts.csv',index=False)
week_union_texts.to_csv('week_texts.csv',index=False)

Повторяем тот же алгоритм с заголовками

In [26]:
def title_union(arr):
    a=str()
    for i in range(len(arr['title'])):
        a+=arr['title'][i]
        a+='\t'
    return a

In [27]:
month_union_titles=pd.DataFrame(columns=['date','title'])
for i in range(len(month_sets)):
    new_row={'date':month_sets[i]['date'][0],'title':title_union(month_sets[i])}
    month_union_titles=month_union_titles.append(new_row,ignore_index=True)

In [28]:
week_union_titles=pd.DataFrame(columns=['date','title'])
for i in range(len(week_sets)):
    new_row={'date':week_sets[i]['date'][0],'title':title_union(week_sets[i])}
    week_union_titles=week_union_titles.append(new_row,ignore_index=True)

# Сохраняем заголовки

In [29]:
month_union_titles.to_csv('month_titles.csv',index=False)
week_union_titles.to_csv('week_titles.csv',index=False)